# INFO 1998 Final Project

## Title of Project

### Benjamin Tang and Dylan Tom

In [1]:
#Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

https://www.kaggle.com/austinreese/craigslist-carstrucks-data

In [9]:
#Import Dataset
data = pd.read_csv('vehicles.csv')
df = pd.DataFrame(data)

## Data Preprocessing and Cleaning

In [11]:
df.shape

(426880, 26)

In [22]:
#Show sample dataset
df = df[df['year'].notna()]

In [25]:
df.iloc[42450]

id                                                     7314848129
url             https://orangecounty.craigslist.org/ctd/d/tust...
region                                              orange county
region_url                    https://orangecounty.craigslist.org
price                                                       19990
year                                                       2021.0
manufacturer                                                  kia
model                                            soul lx wagon 4d
condition                                                    good
cylinders                                                     NaN
fuel                                                          gas
odometer                                                   1943.0
title_status                                                clean
transmission                                                other
VIN                                             KNDJ23AU1M7747914
drive     

## Visualization

In [4]:
#Visualization

## Machine Learning 